In [ ]:
source("../shared.R")
library("MASS")
data("OME")

In [ ]:
dat = subset(OME, OME != "N/A")

In [ ]:
dat = subset(OME, OME != "N/A")
dat$OME = factor(dat$OME) # relabel OME
dat$ID = as.numeric(factor(dat$ID))

In [ ]:
mod_glm = glm(Correct/Trials ~ Age + OME + Loud + Noise, data=dat, weights=Trials, family="binomial")
X = model.matrix(mod_glm)[,-1]

In [ ]:
summary(mod_glm)

### Original non-hierarchial model

In [ ]:
mod_string = " model {
    for (i in 1:length(y)) {
        y[i] ~ dbin(phi[i], n[i])
        logit(phi[i]) = b0 + b[1]*Age[i] + b[2]*OMElow[i] + b[3]*Loud[i] + b[4]*Noiseincoherent[i]
    }
    
    b0 ~ dnorm(0.0, 1.0/5.0^2)
    for (j in 1:4) {
        b[j] ~ dnorm(0.0, 1.0/4.0^2)
    }
} "
data_jags = as.list(as.data.frame(X))
data_jags$y = dat$Correct
data_jags$n = dat$Trials
data_jags$ID = dat$ID

mod = jags_model_compile(str_model_ = mod_string, data_ = data_jags, burnout = 1e3)

mod_sim = jags_model_sample(mod_ = mod, var_names_ = c("b0", "b"), n_iter_ = 1e4)

mod_diag = jags_model_diag(samples_ = mod_sim$samples)

mod_diag$acf_diag

In [ ]:
mod_diag$eff_size

In [ ]:
head(dat)

In [ ]:
mod_hierarchial_str = " model {
    for (i in 1:length(y)) {
        y[i] ~ dbin(phi[i], n[i])
        logit(phi[i]) = a[ID[i]] + b[1]*Age[i] + b[2]*OMElow[i] + b[3]*Loud[i] + b[4]*Noiseincoherent[i]
    }

    for (k in 1:max(ID)){
        a[k] ~ dnorm(mu, prec)
    }
    mu ~ dnorm(0, 1/100.0)
    prec ~ dgamma(1/2, 1/2)
    
    b0 ~ dnorm(0.0, 1.0/5.0^2)
    for (j in 1:4) {
        b[j] ~ dnorm(0.0, 1.0/4.0^2)
    }
} "
data_jags = as.list(as.data.frame(X))
data_jags$y = dat$Correct
data_jags$n = dat$Trials
data_jags$ID = dat$ID

mod_hierarchial = jags_model_compile(str_model_ = mod_hierarchial_str, data_ = data_jags, burnout = 1e3)

In [ ]:
plot(mod_hierarchial_sim$samples_cmb[, c("a[8]", "b[1]", "b[2]")])

In [ ]:

mod_hierarchial_sim = jags_model_sample(mod_ = mod_hierarchial, var_names_ = c("a", "b0", "b"), n_iter_ = 5e3)

mod_hierarchial_diag = jags_model_diag(samples_ = mod_hierarchial_sim$samples)


In [ ]:
dic.samples(model = mod_hierarchial, n.iter = 1e3)